In [1]:
import keras_segmentation

Using TensorFlow backend.


In [2]:
model_name = "resnet50_pspnet"
model = keras_segmentation.models.model_from_name[model_name](n_classes=2,  input_height=384, input_width=576)

InternalError: CUDA runtime implicit initialization on GPU:0 failed. Status: out of memory

In [6]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 384, 576, 3)  0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 390, 582, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 192, 288, 64) 9472        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 192, 288, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [3]:

model.train( 
    train_images =  "/media/new_hdd2/LV-MHP-v2/train/images",
    train_annotations = "/media/new_hdd2/LV-MHP-v2/train/annotations",
    validate=True, 
    val_images="/media/new_hdd2/LV-MHP-v2/val/images", 
    val_annotations="/media/new_hdd2/LV-MHP-v2/val/annotations",
    val_batch_size=2 , 
    checkpoints_path = "/media/new_hdd2/seg_net/{0}/{0}".format(model_name), 
    epochs=50,
    steps_per_epoch=2000,
    optimizer_name='adadelta',
    verify_dataset=False,
    auto_resume_checkpoint=True,
)

Loading the weights from latest checkpoint  /media/new_hdd2/seg_net/resnet50_pspnet/resnet50_pspnet.9
Starting Epoch  0
Epoch 1/1
2000/2000 [==============================] - 239s 119ms/step - loss: 0.1858 - acc: 0.9225 - val_loss: 0.2332 - val_acc: 0.9055
saved  /media/new_hdd2/seg_net/resnet50_pspnet/resnet50_pspnet.model.0
Finished Epoch 0
Starting Epoch  1
Epoch 1/1
2000/2000 [==============================] - 231s 115ms/step - loss: 0.1776 - acc: 0.9260 - val_loss: 0.2088 - val_acc: 0.9150
saved  /media/new_hdd2/seg_net/resnet50_pspnet/resnet50_pspnet.model.1
Finished Epoch 1
Starting Epoch  2
Epoch 1/1
2000/2000 [==============================] - 232s 116ms/step - loss: 0.1740 - acc: 0.9280 - val_loss: 0.2686 - val_acc: 0.8884
saved  /media/new_hdd2/seg_net/resnet50_pspnet/resnet50_pspnet.model.2
Finished Epoch 2
Starting Epoch  3
Epoch 1/1
2000/2000 [==============================] - 233s 117ms/step - loss: 0.2038 - acc: 0.9148 - val_loss: 0.2243 - val_acc: 0.9040
saved  /media/

2000/2000 [==============================] - 232s 116ms/step - loss: 0.1319 - acc: 0.9456 - val_loss: 0.2631 - val_acc: 0.9109
saved  /media/new_hdd2/seg_net/resnet50_pspnet/resnet50_pspnet.model.33
Finished Epoch 33
Starting Epoch  34
Epoch 1/1
2000/2000 [==============================] - 232s 116ms/step - loss: 0.1311 - acc: 0.9459 - val_loss: 0.2472 - val_acc: 0.9128
saved  /media/new_hdd2/seg_net/resnet50_pspnet/resnet50_pspnet.model.34
Finished Epoch 34
Starting Epoch  35
Epoch 1/1
2000/2000 [==============================] - 232s 116ms/step - loss: 0.1280 - acc: 0.9470 - val_loss: 0.2349 - val_acc: 0.9135
saved  /media/new_hdd2/seg_net/resnet50_pspnet/resnet50_pspnet.model.35
Finished Epoch 35
Starting Epoch  36
Epoch 1/1
2000/2000 [==============================] - 232s 116ms/step - loss: 0.1242 - acc: 0.9488 - val_loss: 0.2137 - val_acc: 0.9221
saved  /media/new_hdd2/seg_net/resnet50_pspnet/resnet50_pspnet.model.36
Finished Epoch 36
Starting Epoch  37
Epoch 1/1
2000/2000 [=====

In [2]:
import json
model_name = "resnet50_pspnet"
with open("./{0}/{0}_config.json".format(model_name), "r") as read_file:
    model_config = json.load(read_file)
ckpt = keras_segmentation.train.find_latest_checkpoint("./{0}/{0}".format(model_name))

In [3]:
model = keras_segmentation.pretrained.model_from_checkpoint_path(model_config,ckpt)

In [4]:
import skimage
import glob
import os
import numpy as np
from keras_segmentation.data_utils.data_loader import get_image_arr
path = "/media/new_hdd2/instance-level_human_parsing/Test"
fnames = sorted(glob.glob(os.path.join(path,"*.jpg")))
images = []
for f in fnames:
    images.append(skimage.io.imread(f))
outs = []
for f in fnames:
    outs.append(model.predict_segmentation(inp=f))

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
for inp,otp in zip(images,outs):
    f, axarr = plt.subplots(1,2)
    axarr[0].imshow(skimage.transform.resize(inp,otp.shape,anti_aliasing=True))
    axarr[1].imshow(otp, cmap='Greys',  interpolation='nearest')

/home/avantariml/.local/lib/python3.5/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/home/avantariml/.local/lib/python3.5/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


In [92]:
import cv2
for inp,otp in zip(images,outs):
    mask = skimage.transform.resize(otp,inp.shape[:2],preserve_range=True)
    #mask = cv2.resize(otp,inp.shape[:2])
    im = np.copy(inp)
    im[mask==0]=0
    f, axarr = plt.subplots(1,2)
    axarr[0].imshow(im)
    axarr[1].imshow(inp)

/home/avantariml/.local/lib/python3.5/site-packages/ipykernel_launcher.py:4: DeprecationWarning: Both axis > a.ndim and axis < -a.ndim - 1 are deprecated and will raise an AxisError in the future.
  after removing the cwd from sys.path.


error: OpenCV(4.1.0) /io/opencv/modules/imgproc/src/resize.cpp:3596: error: (-215:Assertion failed) func != 0 in function 'resize'
